# Exploración del ML Detector

Este notebook usa el código del proyecto (sin duplicarlo) para:
- Instanciar el `ThreatDetector`
- Generar datos sintéticos (normal/anómalo)
- Entrenar y evaluar
- Visualizar KMeans, LOF y OneClassSVM en 2D (PCA)

Requisitos de dev (local): `pip install -r applications/ml-detector/requirements-dev.txt`


In [ ]:
# Configurar ruta para importar módulos del proyecto
import sys, os
from pathlib import Path

# Asume que este notebook vive en applications/ml-detector/notebooks
nb_dir = Path.cwd()
app_dir = nb_dir.parent  # applications/ml-detector
sys.path.insert(0, str(app_dir))

# Desactivar entrenamiento/baseline automáticos para exploración reproducible
os.environ['TRAINING_ENABLED'] = 'false'
os.environ['BASELINE_ENABLED'] = 'false'
os.environ.setdefault('MODEL_PATH', '/tmp/models')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline


In [ ]:
# Importar el detector y dependencias
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA

from detector import ThreatDetector

detector = ThreatDetector()
detector.is_trained


In [ ]:
# Generar datos sintéticos: normal y anomalías sencillas
def gen_normal(n=300, seed=42):
    rng = np.random.default_rng(seed)
    return [
        {
            'packets_per_second': float(rng.normal(50, 10)),
            'bytes_per_second': float(rng.normal(50000, 10000)),
            'unique_ips': int(rng.integers(5, 20)),
            'unique_ports': int(rng.integers(3, 10)),
            'tcp_ratio': float(rng.uniform(0.6, 0.8)),
            'syn_packets': int(rng.integers(10, 50)),
        } for _ in range(n)
    ]

def gen_anomalies(n=60, seed=123):
    rng = np.random.default_rng(seed)
    data = []
    for _ in range(n):
        mode = rng.integers(0, 3)
        if mode == 0:  # DDoS-like
            d = {'packets_per_second': float(rng.normal(2000, 200)), 'bytes_per_second': float(rng.normal(2e6, 2e5)), 'unique_ips': int(rng.integers(1, 5)), 'unique_ports': int(rng.integers(1, 3)), 'tcp_ratio': float(rng.uniform(0.7, 1.0)), 'syn_packets': int(rng.integers(500, 1200))}
        elif mode == 1:  # Port-scan-like
            d = {'packets_per_second': float(rng.normal(200, 30)), 'bytes_per_second': float(rng.normal(1e5, 2e4)), 'unique_ips': int(rng.integers(1, 3)), 'unique_ports': int(rng.integers(30, 200)), 'tcp_ratio': float(rng.uniform(0.4, 0.9)), 'syn_packets': int(rng.integers(50, 150))}
        else:  # Exfil-like
            d = {'packets_per_second': float(rng.normal(120, 20)), 'bytes_per_second': float(rng.normal(6e6, 5e5)), 'unique_ips': int(rng.integers(1, 4)), 'unique_ports': int(rng.integers(1, 5)), 'tcp_ratio': float(rng.uniform(0.85, 1.0)), 'syn_packets': int(rng.integers(20, 60))}
        data.append(d)
    return data

normal = gen_normal(400)
anoms = gen_anomalies(80)
len(normal), len(anoms)


In [ ]:
# Alimentar normal al detector y entrenar
for d in normal:
    detector.extract_features(d)
detector.train_models()
detector.is_trained


In [ ]:
# Preparar matriz de características y etiquetas
def to_vec(d):
    return [
        d.get('packets_per_second', 0),
        d.get('bytes_per_second', 0),
        d.get('unique_ips', 0),
        d.get('unique_ports', 0),
        d.get('tcp_ratio', 0.5),
        d.get('syn_packets', 0),
    ]

Xn = np.array([to_vec(d) for d in normal])
Xa = np.array([to_vec(d) for d in anoms])
X = np.vstack([Xn, Xa])
y = np.array([0]*len(Xn) + [1]*len(Xa))  # 0 normal, 1 anómalo

# Proyección 2D con PCA (sobre datos escalados del detector)
Xs = detector.scaler.transform(X) if hasattr(detector.scaler, 'mean_') else X
pca = PCA(n_components=2, random_state=0)
Z = pca.fit_transform(Xs)
Z[:3]


In [ ]:
# Visualización básica: normal vs anomalías
plt.figure(figsize=(6,5))
sns.scatterplot(x=Z[:,0], y=Z[:,1], hue=y, palette={0: 'tab:blue', 1: 'tab:red'}, s=30)
plt.title('Proyección PCA: normal (azul) vs anomalías (rojo)')
plt.xlabel('PC1'); plt.ylabel('PC2'); plt.legend(title='label');
plt.show()


In [ ]:
# KMeans: asignaciones de cluster y centros proyectados
labels = detector.kmeans.predict(Xs)
centers_6d = detector.kmeans.cluster_centers_
centers_2d = pca.transform(centers_6d)

plt.figure(figsize=(6,5))
sns.scatterplot(x=Z[:,0], y=Z[:,1], hue=labels, palette='tab10', s=20, legend=False)
plt.scatter(centers_2d[:,0], centers_2d[:,1], c='black', s=120, marker='x', label='centros')
plt.title('KMeans clusters (PCA)')
plt.xlabel('PC1'); plt.ylabel('PC2'); plt.legend();
plt.show()


In [ ]:
# LOF y One-Class SVM: mapa de calor de score sobre la grilla 2D
def grid_scores(model_decision_fn, Z_data, steps=120):
    x_min, x_max = Z_data[:,0].min()-1, Z_data[:,0].max()+1
    y_min, y_max = Z_data[:,1].min()-1, Z_data[:,1].max()+1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, steps), np.linspace(y_min, y_max, steps))
    grid = np.c_[xx.ravel(), yy.ravel()]
    # Volver de PCA->6D aproximado: usamos inversa de PCA y luego inversa de scaler
    Xs_grid = pca.inverse_transform(grid)
    # Decision en espacio 6D escalado
    zz = model_decision_fn(Xs_grid).reshape(xx.shape)
    return xx, yy, zz

# LOF: decision_function (positivos ~ normal, negativos ~ outlier)
xx, yy, zz_lof = grid_scores(lambda Xs_: detector.lof.decision_function(Xs_), Z)
plt.figure(figsize=(6,5))
plt.contourf(xx, yy, zz_lof, levels=20, cmap='coolwarm', alpha=0.6)
plt.scatter(Z[:,0], Z[:,1], c=y, cmap='bwr', s=15, edgecolor='k', alpha=0.6)
plt.title('LOF decision function (PCA)')
plt.xlabel('PC1'); plt.ylabel('PC2');
plt.show()

# One-Class SVM
xx, yy, zz_svm = grid_scores(lambda Xs_: detector.svm.decision_function(Xs_), Z)
plt.figure(figsize=(6,5))
plt.contourf(xx, yy, zz_svm, levels=20, cmap='PiYG', alpha=0.6)
plt.scatter(Z[:,0], Z[:,1], c=y, cmap='bwr', s=15, edgecolor='k', alpha=0.6)
plt.title('One-Class SVM decision function (PCA)')
plt.xlabel('PC1'); plt.ylabel('PC2');
plt.show()


In [ ]:
# Prueba de inferencia usando el endpoint interno del detector
samples = [normal[0], anoms[0]]
for s in samples:
    print('input:', s)
    print('detect:', detector.detect(s))
    print('-'*60)
